<a href="https://colab.research.google.com/github/castillosebastian/NLU_legal_domain/blob/master/NLU_Semantica_Vectorial_en_Textos_Legales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantica Vectorial en Textos Legales: Exploración

In [2]:
# Libraries, tools and thanks 
import bs4
import nltk
import numpy
import pandas as pd
import io
import requests
from google.colab import drive
import numpy as np 
import json

In [3]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [4]:
%cd  'drive/My Drive/Colab Notebooks/data'

/content/drive/My Drive/Colab Notebooks/data


In [5]:
%ls

corpus_fallosmetdat.json  tbdoctrina.json  tbmetdat.json
corpus_textosfallos.json  tbfallos.json


# Lectura de tablas de datos primarios

In [6]:
with open('tbfallos.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbfallos = json.loads(data1)
tbfallos = pd.DataFrame.from_dict(tbfallos)

In [7]:
tbfallos['textos_fallo'][1]

'#TEXTO COMPLETO#A C U E R D O#En la ciudad de La Plata, a 15 de mayo de 2019, habiéndose establecido, de conformidad con lo dispuesto en el Acuerdo 2078, que deberá observarse el siguiente orden de votación: doctores Soria, de Lázzari, Genoud, Kogan, se reúnen los señores Jueces de la Suprema Corte de Justicia en acuerdo ordinario para pronunciar sentencia definitiva en la causa C. 122.321, "F., R.. Determinación de la capacidad jurídica".#A N T E C E D E N T E S#La Sala I de la Cámara de Apelación en lo Civil y Comercial de San Isidro confirmó el fallo de primera instancia que, en los autos sobre determinación de la capacidad jurídica del señor R. F., resolvió aprobar la rendición de cuentas efectuada por la Curaduría Oficial de Alienados (v. fs. 689 y 660, respectivamente).#Se interpuso, por la titular de la Asesoría de Incapaces n° 2 departamental, recurso extraordinario de inaplicabilidad de ley (v. fs. 705/711).#Oído el señor Procurador General, dictada la providencia de autos y 

In [8]:
with open('tbmetdat.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbmetdat = json.loads(data1)
tbmetdat = pd.DataFrame.from_dict(tbmetdat)

In [9]:
tbmetdat[tbmetdat.nro_causa == 122321]

,nro_causa,fres,textos_raw_len,materia,tipo_fallo,tribunal_emisor,caratula,caratula_pub,magistrados_votantes,tribunal_origen,bnro,magistrados,sumario,voces
1,122321,2019-05-15,160,CIVIL Y COMERCIAL,Sentencia Definitiva,SUPREMA CORTE DE JUSTICIA DE LA PROVINCIA (SCBA),"Figueredo, Rafael s/ Determinación de la capac...","F. ,R. s/ Determinación de la capacidad jurídica",Soria-de Lázzari-Genoud-Kogan,CAMARA DE APELACION EN LO CIVIL Y COMERCIAL SA...,158.0,Soria - de Lázzari - Genoud - Kogan.,\nSentencia-Fundamentación. Rendición de cuen...,"RENDICION DE CUENTAS,APROBACION,SENTENCIA,FUND..."


In [10]:
tbmetdat.loc[1]

nro_causa                                                          122321
fres                                                           2019-05-15
textos_raw_len                                                        160
materia                                                 CIVIL Y COMERCIAL
tipo_fallo                                           Sentencia Definitiva
tribunal_emisor          SUPREMA CORTE DE JUSTICIA DE LA PROVINCIA (SCBA)
caratula                Figueredo, Rafael s/ Determinación de la capac...
caratula_pub             F. ,R. s/ Determinación de la capacidad jurídica
magistrados_votantes                        Soria-de Lázzari-Genoud-Kogan
tribunal_origen         CAMARA DE APELACION EN LO CIVIL Y COMERCIAL SA...
bnro                                                                  158
magistrados                          Soria - de Lázzari - Genoud - Kogan.
sumario                  \nSentencia-Fundamentación. Rendición de cuen...
voces                   RENDICION DE C

In [11]:
tbmetdat.loc[1].sumario

' \nSentencia-Fundamentación. Rendición de cuentas-Aprobación. \nLa Suprema Corte de Justicia de la Provincia de Buenos Aires revocó la sentencia que tubo por \naprobada  la rendición  de  cuentas  presentada  por  el  Curador  Oficial  en  el  marco  del  proceso de \ndeterminación de la capacidad, considerando la falta de fundamentación suficiente del fallo y de \nadecuación a los hechos probados en la causa. '

In [12]:
with open('tbdoctrina.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbdoctrina = json.loads(data1)
tbdoctrina = pd.DataFrame.from_dict(tbdoctrina)

In [13]:
tbdoctrina.head()

,nro_causa,doctrina_nro,doctrina
0,122514,1,El recurso de inaplicabilidad de ley no consti...
1,122514,2,Los juicios de divorcio en trámite al momento ...
2,122514,3,Cuando en la causa sobre divorcio las partes n...
3,122514,4,El artículo 7 del nuevo Código Civil y Comerci...
4,122514,5,La aplicación inmediata de las normas se disti...


In [14]:
tbdoctrina[tbdoctrina.nro_causa == 122321].doctrina

10    Debe ser revocado el fallo que, como único sus...
Name: doctrina, dtype: object

# Procesamiento con Stanza (StanfordNLP)

- https://github.com/stanfordnlp/stanza

In [15]:
! pip install stanza

     |████████████████████████████████| 235kB 1.4MB/s 


In [60]:
import stanza
stanza.download('es') # download English model
nlp = stanza.Pipeline('es') # initialize Spanish neural pipeline
doc = nlp("Alberto Fernandez es presidente de Argetina") # run annotation over a sentence

2020-10-23 15:36:44 INFO: Downloading default packages for language: es (Spanish)...
2020-10-23 15:36:45 INFO: File exists: /root/stanza_resources/es/default.zip.
2020-10-23 15:36:52 INFO: Finished downloading models and saved to /root/stanza_resources.
2020-10-23 15:36:52 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2020-10-23 15:36:52 INFO: Use device: cpu
2020-10-23 15:36:52 INFO: Loading: tokenize
2020-10-23 15:36:52 INFO: Loading: mwt
2020-10-23 15:36:52 INFO: Loading: pos
2020-10-23 15:36:53 INFO: Loading: lemma
2020-10-23 15:36:53 INFO: Loading: depparse
2020-10-23 15:36:54 INFO: Loading: ner
2020-10-23 15:36:55 INFO: Done loading processors!


In [ ]:
doc

# Tokenizacion y Segmentacion de sentencias

## Método por REGEX

In [ ]:
import re
def split_into_sentences(text):       
    text = text.replace("#","<stop>")    
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences
split_into_sentences(tbfallos['textos_fallo'][1])

## Método Stanza

In [66]:
text = tbfallos['textos_fallo'][1]
text = text.replace("#","\n\n")

In [ ]:
nlp = stanza.Pipeline(lang='es', processors='tokenize', tokenize_no_ssplit=True)
doc = nlp(text)
for i, sentence in enumerate(doc.sentences):
    print(f'====== Sentence {i+1} tokens =======')
    print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

In [ ]:
doc.sentences[11]

# POS y Ufeat

In [ ]:
# nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos')
# print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in doc.sentences for word in sent.words], sep='\n')

# Entidades

In [ ]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,ner')
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')

In [69]:
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')


# Imprimo Palabra

In [54]:
word = doc.sentences[0].words[0]
print(word)

{
  "id": 1,
  "text": "TEXTO",
  "misc": "start_char=2|end_char=7"
}
